In [4]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

Using cache found in /home/zakaria/.cache/torch/hub/pytorch_vision_v0.10.0
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zakaria/miniconda3/envs/photoen/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [6]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [7]:
pred.shape

torch.Size([1, 512, 1, 1])

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [ ]:
# import os 
# os.chdir('envs')

In [6]:
from envs.env_dataloader import create_dataloaders
train,test = create_dataloaders()

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [64]:
f=train.dataset.encoded_source

In [66]:
from envs.new_edit_photo import PhotoEditor

editor = PhotoEditor()
parameters = torch.tensor([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])
parameters = torch.stack([parameters for i in range(64)])

In [68]:
parameters.shape

torch.Size([64, 12])

In [96]:
for i,j,k,l in train:
    print(i.dtype)
    enhanced = i.permute(0,2,3,1)/255.0
    print(enhanced.dtype)
    enhanced = editor(enhanced,parameters)
    enhanced = enhanced.permute(0,3,1,2)
    enhanced =torch.flatten(enhanced,start_dim=1, end_dim=-1)

    target = torch.flatten(j,start_dim=1, end_dim=-1)/255.0
    print(enhanced.dtype)
    rmse = enhanced-target
    rmse = torch.pow(rmse,2).mean(1)
    rmse = torch.sqrt(rmse)
    break
print(rmse)

torch.uint8
torch.float32
torch.float64
tensor([0.0707, 0.0590, 0.0879, 0.0591, 0.0504, 0.1089, 0.1085, 0.0698, 0.1076,
        0.1245, 0.1206, 0.0845, 0.1030, 0.1212, 0.2703, 0.0550, 0.0227, 0.0917,
        0.0648, 0.1369, 0.0340, 0.1386, 0.0741, 0.2462, 0.0919, 0.0667, 0.1040,
        0.0868, 0.0984, 0.1001, 0.1288, 0.0719, 0.1388, 0.1606, 0.0406, 0.0617,
        0.0638, 0.0642, 0.0689, 0.1094, 0.0588, 0.0838, 0.0435, 0.1866, 0.0658,
        0.0560, 0.1915, 0.0764, 0.1464, 0.0744, 0.0416, 0.0497, 0.1809, 0.1085,
        0.1759, 0.1005, 0.0565, 0.0777, 0.1438, 0.0625, 0.1568, 0.1311, 0.1586,
        0.0816], dtype=torch.float64)
